# Grid Search Optimization Results Analysis

This notebook analyzes the results from grid search parameter optimization for the MovingAverageCrossover strategy.

## Objectives
1. Identify top-performing parameter combinations
2. Understand parameter sensitivity and interactions
3. Visualize performance distributions and correlations
4. Export optimal parameters for walk-forward validation

## Analysis Workflow
1. **Data Loading**: Load grid search results CSV
2. **Top Performers**: Display best parameter sets
3. **Parameter Heatmaps**: Visualize parameter interactions
4. **Performance Distributions**: Analyze PnL, Sharpe, win rate distributions
5. **Correlation Analysis**: Identify relationships between metrics
6. **Parameter Sensitivity**: Understand which parameters matter most
7. **Export**: Save top parameters for validation

## Prerequisites
- Grid search results CSV from `optimization/grid_search.py`
- Python packages: pandas, matplotlib, seaborn, numpy


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
from typing import List, Dict, Tuple
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style("whitegrid")
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Imports and setup complete")


## Configuration

Specify the path to your grid search results CSV file.

**Expected file location:**
- Default: `optimization/results/grid_search_results.csv`
- Or specify custom path from grid search run

**File structure:**
- Parameter columns: fast_period, slow_period, crossover_threshold_pips, etc.
- Metric columns: total_pnl, sharpe_ratio, win_rate, max_drawdown, etc.
- Metadata columns: run_id, status, rank, objective_value


In [ ]:
# Configuration: Update this path to your grid search results
RESULTS_CSV = "optimization/results/grid_search_results.csv"
OBJECTIVE = "total_pnl"  # Objective function used in optimization

# Load results
try:
    df = pd.read_csv(RESULTS_CSV)
    print(f"✅ Loaded {len(df)} backtest results from {RESULTS_CSV}")
except FileNotFoundError:
    print(f"❌ Error: File not found: {RESULTS_CSV}")
    print("Please update RESULTS_CSV path or run grid search first.")
    raise

# Display basic info
print(f"\nDataset shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"\nColumns: {list(df.columns)}")

# Check for failed runs
failed_count = len(df[df['status'] != 'completed'])
if failed_count > 0:
    print(f"\n⚠️  Warning: {failed_count} failed runs detected")
    print(f"Failed runs will be excluded from analysis")

# Filter to completed runs only
df_completed = df[df['status'] == 'completed'].copy()

# Guard against empty dataset
if len(df_completed) == 0:
    print("\n❌ Error: No completed runs found in the dataset.")
    print("Please check your grid search results or run the optimization again.")
    raise ValueError("No completed runs available for analysis")

print(f"\n✅ Analysis dataset: {len(df_completed)} completed runs")

# Ensure numeric dtype coercion for expected numeric columns
numeric_columns = [
    'fast_period', 'slow_period', 'crossover_threshold_pips', 'stop_loss_pips', 'take_profit_pips',
    'trailing_stop_activation_pips', 'trailing_stop_distance_pips', 'dmi_period',
    'stoch_period_k', 'stoch_period_d', 'stoch_bullish_threshold', 'stoch_bearish_threshold',
    'total_pnl', 'sharpe_ratio', 'win_rate', 'max_drawdown', 'trade_count', 'profit_factor', 'expectancy'
]

for col in numeric_columns:
    if col in df_completed.columns:
        df_completed[col] = pd.to_numeric(df_completed[col], errors='coerce')

print("✅ Numeric columns coerced to proper dtypes")


## Data Overview

Let's examine the structure and summary statistics of the optimization results.


In [ ]:
# Display first few rows
print("First 5 rows:")
print(df_completed.head())

# Summary statistics for metrics
print("\n" + "="*80)
print("Summary Statistics (Metrics)")
print("="*80)
metric_cols = ['total_pnl', 'sharpe_ratio', 'win_rate', 'max_drawdown', 'trade_count', 
               'profit_factor', 'expectancy']
print(df_completed[metric_cols].describe())

# Summary statistics for parameters
print("\n" + "="*80)
print("Summary Statistics (Parameters)")
print("="*80)
param_cols = ['fast_period', 'slow_period', 'crossover_threshold_pips', 
              'stop_loss_pips', 'take_profit_pips']
print(df_completed[param_cols].describe())


## Top 10 Parameter Sets

Best-performing parameter combinations ranked by total PnL.

**Key Metrics:**
- **Total PnL**: Absolute profit/loss in base currency
- **Sharpe Ratio**: Risk-adjusted return (higher is better, >1.5 is good)
- **Win Rate**: Percentage of profitable trades
- **Max Drawdown**: Largest peak-to-trough decline (lower is better)
- **Trade Count**: Number of trades executed (need >20 for statistical significance)


In [ ]:
# Select columns to display
display_cols = [
    'rank', 'fast_period', 'slow_period', 'crossover_threshold_pips',
    'stop_loss_pips', 'take_profit_pips', 'total_pnl', 'sharpe_ratio', 
    'win_rate', 'max_drawdown', 'trade_count'
]

# Get top 10 by total_pnl descending (or OBJECTIVE if configured)
ranking_column = OBJECTIVE if OBJECTIVE in df_completed.columns else 'total_pnl'
top_10 = df_completed.nlargest(10, ranking_column)[display_cols]

print(f"Top 10 Parameter Sets (Ranked by {ranking_column} Descending)")
print("="*120)
print(top_10.to_string(index=False))

# Highlight best result
best = top_10.iloc[0]
print("\n" + "="*120)
print("🏆 BEST RESULT:")
print("="*120)
for col in display_cols:
    print(f"{col:30s}: {best[col]}")


## Parameter Heatmaps

Heatmaps show how PnL varies with different parameter combinations.
Each heatmap is a 2D slice showing the interaction between two parameters.

**Key Parameter Pairs:**
1. **Fast Period vs Slow Period**: Core MA crossover parameters
2. **Stop Loss vs Take Profit**: Risk/reward relationship
3. **Crossover Threshold vs DMI Period**: Filter interaction
4. **Stochastic Bullish vs Bearish Threshold**: Momentum filter tuning

**Interpretation:**
- **Hot colors (red/orange)**: High PnL (good)
- **Cool colors (blue/purple)**: Low PnL (poor)
- **Patterns**: Look for "sweet spots" (concentrated high-PnL regions)


In [ ]:
# Create pivot table for fast_period vs slow_period
pivot_fast_slow = df_completed.pivot_table(
    values='total_pnl',
    index='slow_period',
    columns='fast_period',
    aggfunc='mean'  # Average PnL if multiple runs with same combination
)

# Create heatmap
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(
    pivot_fast_slow,
    annot=True,
    fmt='.0f',
    cmap='RdYlGn',
    center=0,
    cbar_kws={'label': 'Total PnL ($)'},
    ax=ax
)
ax.set_title('PnL Heatmap: Fast Period vs Slow Period', fontsize=14, fontweight='bold')
ax.set_xlabel('Fast Period', fontsize=12)
ax.set_ylabel('Slow Period', fontsize=12)
plt.tight_layout()
plt.show()

print(f"Best combination: Fast={pivot_fast_slow.stack().idxmax()[1]}, Slow={pivot_fast_slow.stack().idxmax()[0]}")
print(f"Best PnL: ${pivot_fast_slow.max().max():.2f}")


In [ ]:
# Create pivot table for stop_loss_pips vs take_profit_pips
pivot_sl_tp = df_completed.pivot_table(
    values='total_pnl',
    index='take_profit_pips',
    columns='stop_loss_pips',
    aggfunc='mean'
)

# Create heatmap
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(
    pivot_sl_tp,
    annot=True,
    fmt='.0f',
    cmap='RdYlGn',
    center=0,
    cbar_kws={'label': 'Total PnL ($)'},
    ax=ax
)
ax.set_title('PnL Heatmap: Stop Loss vs Take Profit', fontsize=14, fontweight='bold')
ax.set_xlabel('Stop Loss (pips)', fontsize=12)
ax.set_ylabel('Take Profit (pips)', fontsize=12)
plt.tight_layout()
plt.show()

# Calculate optimal risk/reward ratio with division by zero guard
best_sl, best_tp = pivot_sl_tp.stack().idxmax()
if best_sl > 0:
    risk_reward_ratio = best_tp / best_sl
    print(f"Best combination: SL={best_sl} pips, TP={best_tp} pips")
    print(f"Risk/Reward Ratio: 1:{risk_reward_ratio:.2f}")
    print(f"Best PnL: ${pivot_sl_tp.max().max():.2f}")
else:
    print(f"Best combination: SL={best_sl} pips, TP={best_tp} pips")
    print(f"Risk/Reward Ratio: Cannot calculate (SL=0)")
    print(f"Best PnL: ${pivot_sl_tp.max().max():.2f}")


In [ ]:
# Create pivot table for crossover_threshold_pips vs dmi_period
pivot_threshold_dmi = df_completed.pivot_table(
    values='total_pnl',
    index='dmi_period',
    columns='crossover_threshold_pips',
    aggfunc='mean'
)

# Create heatmap
fig, ax = plt.subplots(figsize=(10, 6))
sns.heatmap(
    pivot_threshold_dmi,
    annot=True,
    fmt='.0f',
    cmap='RdYlGn',
    center=0,
    cbar_kws={'label': 'Total PnL ($)'},
    ax=ax
)
ax.set_title('PnL Heatmap: Crossover Threshold vs DMI Period', fontsize=14, fontweight='bold')
ax.set_xlabel('Crossover Threshold (pips)', fontsize=12)
ax.set_ylabel('DMI Period', fontsize=12)
plt.tight_layout()
plt.show()

print(f"Best combination: Threshold={pivot_threshold_dmi.stack().idxmax()[1]} pips, DMI Period={pivot_threshold_dmi.stack().idxmax()[0]}")
print(f"Best PnL: ${pivot_threshold_dmi.max().max():.2f}")


## PnL Distribution Analysis

Histogram showing the distribution of total PnL across all parameter combinations.

**Interpretation:**
- **Shape**: Normal distribution suggests parameters have predictable impact
- **Skewness**: Right-skewed (long tail to right) indicates some parameter sets perform exceptionally well
- **Outliers**: Extreme values may indicate overfitting or data issues
- **Spread**: Wide distribution suggests parameters have significant impact on performance


In [ ]:
# Create PnL distribution histogram
fig, ax = plt.subplots(figsize=(12, 6))

ax.hist(df_completed['total_pnl'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
ax.axvline(df_completed['total_pnl'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: ${df_completed["total_pnl"].mean():.2f}')
ax.axvline(df_completed['total_pnl'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: ${df_completed["total_pnl"].median():.2f}')
ax.axvline(df_completed['total_pnl'].max(), color='orange', linestyle='--', linewidth=2, label=f'Best: ${df_completed["total_pnl"].max():.2f}')

ax.set_xlabel('Total PnL ($)', fontsize=12)
ax.set_ylabel('Frequency (Number of Parameter Sets)', fontsize=12)
ax.set_title('Distribution of Total PnL Across All Parameter Combinations', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print statistics
print(f"PnL Statistics:")
print(f"  Mean:   ${df_completed['total_pnl'].mean():.2f}")
print(f"  Median: ${df_completed['total_pnl'].median():.2f}")
print(f"  Std:    ${df_completed['total_pnl'].std():.2f}")
print(f"  Min:    ${df_completed['total_pnl'].min():.2f}")
print(f"  Max:    ${df_completed['total_pnl'].max():.2f}")
print(f"  Range:  ${df_completed['total_pnl'].max() - df_completed['total_pnl'].min():.2f}")


## Scatter Plot Analysis

Scatter plots reveal relationships between different performance metrics.

**Key Relationships:**
1. **Sharpe Ratio vs Total PnL**: Are high-PnL strategies also risk-efficient?
2. **Win Rate vs Trade Count**: Do more trades lead to higher win rates?
3. **Max Drawdown vs Total PnL**: Is higher PnL achieved at the cost of larger drawdowns?

**Ideal Quadrant:**
- High PnL + High Sharpe (top-right in Sharpe vs PnL plot)
- High Win Rate + Sufficient Trades (top-right in Win Rate vs Trade Count plot)


In [ ]:
# Create scatter plot: Sharpe Ratio vs Total PnL
fig, ax = plt.subplots(figsize=(12, 8))

scatter = ax.scatter(
    df_completed['total_pnl'],
    df_completed['sharpe_ratio'],
    c=df_completed['win_rate'],
    s=df_completed['trade_count'] * 2,  # Size by trade count
    alpha=0.6,
    cmap='viridis',
    edgecolors='black',
    linewidth=0.5
)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Win Rate', fontsize=12)

# Highlight top 10 results
top_10_df = df_completed.nsmallest(10, 'rank')
ax.scatter(
    top_10_df['total_pnl'],
    top_10_df['sharpe_ratio'],
    s=200,
    facecolors='none',
    edgecolors='red',
    linewidth=3,
    label='Top 10 Results'
)

# Annotate best result
best_row = df_completed.loc[df_completed['rank'] == 1].iloc[0]
ax.annotate(
    f"Best: ${best_row['total_pnl']:.0f}",
    xy=(best_row['total_pnl'], best_row['sharpe_ratio']),
    xytext=(10, 10),
    textcoords='offset points',
    fontsize=10,
    bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7),
    arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0')
)

ax.set_xlabel('Total PnL ($)', fontsize=12)
ax.set_ylabel('Sharpe Ratio', fontsize=12)
ax.set_title('Sharpe Ratio vs Total PnL (bubble size = trade count, color = win rate)', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Interpretation:")
print("- Top-right quadrant: High PnL + High Sharpe (ideal)")
print("- Larger bubbles: More trades (better statistical significance)")
print("- Yellow/green colors: Higher win rates")


In [ ]:
# Create scatter plot: Win Rate vs Trade Count
fig, ax = plt.subplots(figsize=(12, 8))

scatter = ax.scatter(
    df_completed['trade_count'],
    df_completed['win_rate'],
    c=df_completed['total_pnl'],
    s=100,
    alpha=0.6,
    cmap='RdYlGn',
    edgecolors='black',
    linewidth=0.5
)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Total PnL ($)', fontsize=12)

# Highlight top 10
ax.scatter(
    top_10_df['trade_count'],
    top_10_df['win_rate'],
    s=200,
    facecolors='none',
    edgecolors='red',
    linewidth=3,
    label='Top 10 Results'
)

# Add reference lines
ax.axhline(0.5, color='gray', linestyle='--', alpha=0.5, label='50% Win Rate')
ax.axvline(20, color='gray', linestyle='--', alpha=0.5, label='Min 20 Trades')

ax.set_xlabel('Trade Count', fontsize=12)
ax.set_ylabel('Win Rate', fontsize=12)
ax.set_title('Win Rate vs Trade Count (color = total PnL)', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Interpretation:")
print("- Top-right quadrant: High win rate + Many trades (ideal)")
print("- Green points: Profitable parameter sets")
print("- Red points: Unprofitable parameter sets")
print(f"- {len(df_completed[df_completed['trade_count'] < 20])} parameter sets have <20 trades (low statistical significance)")


## Parameter Sensitivity Analysis

Parameter sensitivity shows how much PnL changes when a parameter is varied.

**High Sensitivity Parameters:**
- Large PnL changes with small parameter changes
- Require careful tuning
- Risk of overfitting

**Low Sensitivity Parameters:**
- PnL relatively stable across parameter range
- More robust to parameter variations
- Safer for live trading

**Method:**
For each parameter, we plot average PnL vs parameter value to visualize the sensitivity curve.


In [ ]:
# Calculate average PnL for each parameter value
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Fast Period Sensitivity
fast_sensitivity = df_completed.groupby('fast_period')['total_pnl'].agg(['mean', 'std', 'count'])
axes[0].errorbar(
    fast_sensitivity.index,
    fast_sensitivity['mean'],
    yerr=fast_sensitivity['std'],
    marker='o',
    markersize=8,
    capsize=5,
    capthick=2,
    linewidth=2,
    color='steelblue'
)
axes[0].set_xlabel('Fast Period', fontsize=12)
axes[0].set_ylabel('Average Total PnL ($)', fontsize=12)
axes[0].set_title('Fast Period Sensitivity', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(0, color='red', linestyle='--', alpha=0.5)

# Slow Period Sensitivity
slow_sensitivity = df_completed.groupby('slow_period')['total_pnl'].agg(['mean', 'std', 'count'])
axes[1].errorbar(
    slow_sensitivity.index,
    slow_sensitivity['mean'],
    yerr=slow_sensitivity['std'],
    marker='o',
    markersize=8,
    capsize=5,
    capthick=2,
    linewidth=2,
    color='darkorange'
)
axes[1].set_xlabel('Slow Period', fontsize=12)
axes[1].set_ylabel('Average Total PnL ($)', fontsize=12)
axes[1].set_title('Slow Period Sensitivity', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].axhline(0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

# Calculate sensitivity metrics
fast_pnl_range = fast_sensitivity['mean'].max() - fast_sensitivity['mean'].min()
slow_pnl_range = slow_sensitivity['mean'].max() - slow_sensitivity['mean'].min()

print(f"Fast Period Sensitivity: PnL range = ${fast_pnl_range:.2f}")
print(f"Slow Period Sensitivity: PnL range = ${slow_pnl_range:.2f}")
print(f"\nMost sensitive: {'Fast Period' if fast_pnl_range > slow_pnl_range else 'Slow Period'}")


In [ ]:
# Calculate sensitivity for filter parameters
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Crossover Threshold Sensitivity
threshold_sensitivity = df_completed.groupby('crossover_threshold_pips')['total_pnl'].agg(['mean', 'std'])
axes[0, 0].errorbar(
    threshold_sensitivity.index,
    threshold_sensitivity['mean'],
    yerr=threshold_sensitivity['std'],
    marker='o',
    markersize=8,
    capsize=5,
    linewidth=2,
    color='purple'
)
axes[0, 0].set_xlabel('Crossover Threshold (pips)', fontsize=11)
axes[0, 0].set_ylabel('Average Total PnL ($)', fontsize=11)
axes[0, 0].set_title('Crossover Threshold Sensitivity', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].axhline(0, color='red', linestyle='--', alpha=0.5)

# DMI Period Sensitivity
dmi_sensitivity = df_completed.groupby('dmi_period')['total_pnl'].agg(['mean', 'std'])
axes[0, 1].errorbar(
    dmi_sensitivity.index,
    dmi_sensitivity['mean'],
    yerr=dmi_sensitivity['std'],
    marker='o',
    markersize=8,
    capsize=5,
    linewidth=2,
    color='teal'
)
axes[0, 1].set_xlabel('DMI Period', fontsize=11)
axes[0, 1].set_ylabel('Average Total PnL ($)', fontsize=11)
axes[0, 1].set_title('DMI Period Sensitivity', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].axhline(0, color='red', linestyle='--', alpha=0.5)

# Stochastic Bullish Threshold Sensitivity
stoch_bull_sensitivity = df_completed.groupby('stoch_bullish_threshold')['total_pnl'].agg(['mean', 'std'])
axes[1, 0].errorbar(
    stoch_bull_sensitivity.index,
    stoch_bull_sensitivity['mean'],
    yerr=stoch_bull_sensitivity['std'],
    marker='o',
    markersize=8,
    capsize=5,
    linewidth=2,
    color='green'
)
axes[1, 0].set_xlabel('Stochastic Bullish Threshold', fontsize=11)
axes[1, 0].set_ylabel('Average Total PnL ($)', fontsize=11)
axes[1, 0].set_title('Stochastic Bullish Threshold Sensitivity', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axhline(0, color='red', linestyle='--', alpha=0.5)

# Stochastic Bearish Threshold Sensitivity
stoch_bear_sensitivity = df_completed.groupby('stoch_bearish_threshold')['total_pnl'].agg(['mean', 'std'])
axes[1, 1].errorbar(
    stoch_bear_sensitivity.index,
    stoch_bear_sensitivity['mean'],
    yerr=stoch_bear_sensitivity['std'],
    marker='o',
    markersize=8,
    capsize=5,
    linewidth=2,
    color='crimson'
)
axes[1, 1].set_xlabel('Stochastic Bearish Threshold', fontsize=11)
axes[1, 1].set_ylabel('Average Total PnL ($)', fontsize=11)
axes[1, 1].set_title('Stochastic Bearish Threshold Sensitivity', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].axhline(0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

print("Parameter Sensitivity Summary:")
print(f"  Threshold:        ${threshold_sensitivity['mean'].max() - threshold_sensitivity['mean'].min():.2f} range")
print(f"  DMI Period:       ${dmi_sensitivity['mean'].max() - dmi_sensitivity['mean'].min():.2f} range")
print(f"  Stoch Bullish:    ${stoch_bull_sensitivity['mean'].max() - stoch_bull_sensitivity['mean'].min():.2f} range")
print(f"  Stoch Bearish:    ${stoch_bear_sensitivity['mean'].max() - stoch_bear_sensitivity['mean'].min():.2f} range")


## Correlation Analysis

Correlation matrix shows relationships between parameters and performance metrics.

**Interpretation:**
- **Positive correlation (red)**: Parameter increase leads to metric increase
- **Negative correlation (blue)**: Parameter increase leads to metric decrease
- **No correlation (white)**: Parameter has little impact on metric

**Key Questions:**
- Which parameters most strongly correlate with PnL?
- Are there unexpected correlations between parameters?
- Do filters (DMI, Stochastic) correlate with win rate?


In [ ]:
# Select numeric columns for correlation
corr_cols = [
    'fast_period', 'slow_period', 'crossover_threshold_pips',
    'stop_loss_pips', 'take_profit_pips', 'dmi_period',
    'stoch_period_k', 'stoch_period_d', 'stoch_bullish_threshold', 'stoch_bearish_threshold',
    'trailing_stop_activation_pips', 'trailing_stop_distance_pips',
    'total_pnl', 'sharpe_ratio', 'win_rate', 'max_drawdown', 'trade_count'
]

# Filter to only include columns that exist in the dataset
corr_cols = [col for col in corr_cols if col in df_completed.columns]

# Calculate correlation matrix
corr_matrix = df_completed[corr_cols].corr()

# Create heatmap
fig, ax = plt.subplots(figsize=(14, 12))
sns.heatmap(
    corr_matrix,
    annot=True,
    fmt='.2f',
    cmap='RdBu_r',
    center=0,
    vmin=-1,
    vmax=1,
    square=True,
    linewidths=0.5,
    cbar_kws={'label': 'Correlation Coefficient'},
    ax=ax
)
ax.set_title('Parameter and Metric Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Extract top correlations with PnL
pnl_correlations = corr_matrix['total_pnl'].drop('total_pnl').sort_values(ascending=False)
print("\nTop 5 Positive Correlations with Total PnL:")
print(pnl_correlations.head(5))
print("\nTop 5 Negative Correlations with Total PnL:")
print(pnl_correlations.tail(5))


## Export Top 5 Parameters for Walk-Forward Validation

Export the best 5 parameter sets to JSON format for use in walk-forward validation.

**Output Format:**
```json
[
  {
    "rank": 1,
    "run_id": 42,
    "parameters": {
      "fast_period": 40,
      "slow_period": 200,
      "crossover_threshold_pips": 0.9,
      ...
    },
    "total_pnl": 12500.0,
    "sharpe_ratio": 1.4,
    "win_rate": 0.62
  },
  ...
]
```

**Usage:**
```bash
python optimization/walk_forward.py --params analysis/top_5_params.json
```


In [ ]:
# Helper function for robust boolean parsing
def strtobool(val):
    """Convert string representation of truth to boolean (1, 0, true, false, yes, no, on, off)."""
    if isinstance(val, bool):
        return val
    if isinstance(val, (int, float)):
        return bool(val)
    val = str(val).lower()
    if val in ('1', 'true', 'yes', 'on', 'y', 't'):
        return True
    elif val in ('0', 'false', 'no', 'off', 'n', 'f'):
        return False
    else:
        raise ValueError(f"Cannot convert '{val}' to boolean")

# Get top 5 results
top_5 = df_completed.nsmallest(5, 'rank')

# Build export structure
top_5_export = []
for idx, row in top_5.iterrows():
    param_set = {
        "rank": int(row['rank']),
        "run_id": int(row['run_id']),
        "parameters": {
            "fast_period": int(row['fast_period']),
            "slow_period": int(row['slow_period']),
            "crossover_threshold_pips": float(row['crossover_threshold_pips']),
            "stop_loss_pips": int(row['stop_loss_pips']),
            "take_profit_pips": int(row['take_profit_pips']),
            "trailing_stop_activation_pips": int(row['trailing_stop_activation_pips']),
            "trailing_stop_distance_pips": int(row['trailing_stop_distance_pips']),
            "dmi_enabled": strtobool(row['dmi_enabled']),
            "dmi_period": int(row['dmi_period']),
            "stoch_enabled": strtobool(row['stoch_enabled']),
            "stoch_period_k": int(row['stoch_period_k']),
            "stoch_period_d": int(row['stoch_period_d']),
            "stoch_bullish_threshold": int(row['stoch_bullish_threshold']),
            "stoch_bearish_threshold": int(row['stoch_bearish_threshold']),
        },
        "metrics": {
            "total_pnl": float(row['total_pnl']),
            "sharpe_ratio": float(row['sharpe_ratio']),
            "win_rate": float(row['win_rate']),
            "max_drawdown": float(row['max_drawdown']),
            "trade_count": int(row['trade_count']),
            "profit_factor": float(row['profit_factor']),
        }
    }
    top_5_export.append(param_set)

# Export to JSON
output_path = Path("analysis/top_5_params.json")
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, 'w') as f:
    json.dump(top_5_export, f, indent=2)

print(f"✅ Exported top 5 parameter sets to: {output_path}")
print(f"\nTop 5 Summary:")
for i, params in enumerate(top_5_export, 1):
    print(f"\n{i}. Rank {params['rank']}: PnL=${params['metrics']['total_pnl']:.2f}, Sharpe={params['metrics']['sharpe_ratio']:.2f}")
    print(f"   Fast={params['parameters']['fast_period']}, Slow={params['parameters']['slow_period']}, Threshold={params['parameters']['crossover_threshold_pips']} pips")


## Conclusions and Recommendations

### Key Findings

**Best Parameter Set:**
- Fast Period: [value from top result]
- Slow Period: [value from top result]
- Crossover Threshold: [value from top result] pips
- Stop Loss: [value from top result] pips
- Take Profit: [value from top result] pips
- **Performance**: PnL=$[value], Sharpe=[value], Win Rate=[value]%

**Parameter Sensitivity:**
- **Most sensitive**: [parameter with largest PnL range]
- **Least sensitive**: [parameter with smallest PnL range]
- **Recommendation**: Focus tuning efforts on high-sensitivity parameters

**Filter Effectiveness:**
- DMI Period: [optimal value] shows best results
- Stochastic Thresholds: [optimal values] provide good balance
- Crossover Threshold: [optimal value] pips filters noise effectively

### Next Steps

1. **Walk-Forward Validation** (Critical):
   ```bash
   python optimization/walk_forward.py --params analysis/top_5_params.json --train-months 3 --test-months 1
   ```
   - Validates parameters on out-of-sample data
   - Prevents overfitting
   - Required before live trading

2. **Robustness Testing**:
   - Test top parameters on different forex pairs (GBP/USD, USD/JPY)
   - Test on different time periods (2024 data, 2023 data)
   - Run Monte Carlo simulation with price noise

3. **Fine-Grid Refinement** (Optional):
   - If top results cluster in a region, run fine grid search
   - Zoom into best region with narrower parameter spacing
   - Example: If best fast_period is 40, test [35, 38, 40, 42, 45]

4. **Filter Contribution Analysis**:
   - Run comparative backtests with filters disabled
   - Measure actual PnL contribution of each filter
   - Use `analysis/analyze_filter_effectiveness.py`

5. **Paper Trading**:
   - After walk-forward validation, deploy to paper trading
   - Monitor performance for 1-2 weeks
   - Compare live results to backtest expectations

### Warnings

⚠️ **Overfitting Risk**: These parameters are optimized on historical data. Always validate out-of-sample before live trading.

⚠️ **Market Regime Changes**: Parameters optimized on trending markets may fail in ranging markets (and vice versa).

⚠️ **Statistical Significance**: Ensure top results have sufficient trade count (>20 trades minimum).


In [ ]:
# Interactive parameter exploration
# Modify these values to explore different parameter combinations

EXPLORE_FAST = 50
EXPLORE_SLOW = 150

# Filter results for specific parameter values
filtered = df_completed[
    (df_completed['fast_period'] == EXPLORE_FAST) &
    (df_completed['slow_period'] == EXPLORE_SLOW)
]

if len(filtered) > 0:
    print(f"Results for Fast={EXPLORE_FAST}, Slow={EXPLORE_SLOW}:")
    print(f"Number of combinations: {len(filtered)}")
    print(f"\nBest result with these MA periods:")
    best_filtered = filtered.nsmallest(1, 'rank')
    print(best_filtered[['crossover_threshold_pips', 'stop_loss_pips', 'take_profit_pips', 
                         'total_pnl', 'sharpe_ratio', 'win_rate']].to_string(index=False))
else:
    print(f"No results found for Fast={EXPLORE_FAST}, Slow={EXPLORE_SLOW}")
    print("Try different values or check your grid configuration.")
